In [38]:
import pandas as pd
import numpy as np
# import pandas
import pandas as pd

# import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
data = '/Users/patrickokwir/Desktop/Git_Projects/Ted-Talks-Recommender-System/Data_output/talks.csv'

In [40]:
df = pd.read_csv(data, index_col=0)

In [74]:
df.head(5)

,author,talk,description,likes,views,item
0,Conor Russomanno,a powerful new neurotech tool for augmenting y...,astonishing talk tech demo neurotechnologist c...,4700,157930,astonishing talk tech demo neurotechnologist c...
1,Peter Singer,a modern argument for the rights of animals,prioritize human rights species philosopher pe...,7600,254482,prioritize human rights species philosopher pe...
2,Sahar Zand,why iranians are cutting their hair for woman ...,filmmaker sahar zand vividly explores ongoing ...,1100,393882,filmmaker sahar zand vividly explores ongoing ...
3,Shannon Odell,are solar panels worth it,today many countries solar cheapest form energ...,3700,126251,today many countries solar cheapest form energ...
4,Angus Hervey,why are we so bad at reporting good news,good news rare special broadcast ted stage jou...,1200,415329,good news rare special broadcast ted stage jou...


In [75]:
# find author = 'Simon Sinek' from df
df[df['author'] == 'Simon Sinek']

,author,talk,description,likes,views,item
847,Simon Sinek,how to discover your why in difficult times,coronavirus pandemic taught us relationships d...,6500,2178384,coronavirus pandemic taught us relationships d...
3911,Simon Sinek,why good leaders make you feel safe,makes great leader management theorist simon s...,56400,18806012,makes great leader management theorist simon s...
5124,Simon Sinek,how great leaders inspire action,simon sinek simple powerful model inspirationa...,1800000,62324730,simon sinek simple powerful model inspirationa...


In [42]:
from nltk.corpus import stopwords
import string
stop_words = set(stopwords.words('english'))
df['description'] = df['description'].str.strip()
df['description'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df['description'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.punctuation]))
df['description'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.digits]))

In [43]:
tfidf = TfidfVectorizer(stop_words='english')

In [44]:
# fit and transform 'description' column
tfidf_matrix = tfidf.fit_transform(df['description'])

In [45]:
tfidf_matrix.shape

(5798, 28410)

In [46]:
tfidf.get_feature_names_out()[500:4600]

array(['93', '94year', '95', ..., 'ching', 'chintis', 'chinyanta'],
      dtype=object)

In [47]:
# compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [48]:
cosine_sim.shape

(5798, 5798)

In [49]:
# sort all items using their similarity for each item i, and store the values in dictionary of lists
results = {}
for i in range(len(cosine_sim)):
    results[i] = cosine_sim[i].argsort()[:-11:-1].tolist()

In [50]:
results[1]

[1, 847, 805, 765, 130, 347, 409, 396, 120, 641]

In [51]:
# first split the 'description' so it returns only the item
df['item'] = df['description'].str.split('-').str[0]

In [52]:
# second, create a reverse map of indices and descriptions
indices = pd.Series(df.index, index=df['item']).drop_duplicates()

In [53]:
indices[:2]

item
astonishing talk tech demo neurotechnologist conor russomanno shares work building braincomputer interfaces could enable us control external world minds discusses quickly advancing possibilities field including promise closedloop system could record stimulate brain activity invites neurohacker christian bayerlein onto ted stage fly mindcontrolled drone using biosensing headset                                                                                           0
prioritize human rights species philosopher peter singer dives calls speciesism root widely ignored mistreatment animals around world factory farms producttesting facilities makes case ending commercial exploitation animals food invites everyone reexamine environmental moral weight status quo conversation hosted head ted chris anderson part exclusive ted membership event visit tedcommembership become ted member warning conversation contains images animal cruelty    1
dtype: int64

In [71]:
# now we create a function recomender that will recommend simmilar products, function must take item_id and count as parameters
cosine_sim=cosine_sim

talk_to_search = 'are solar panels worth it'
top_n_results = 3

def recomender(talk_to_search, top_n_results):
   
    count = top_n_results

    id = df.loc[df['talk'] == talk_to_search].index.values[0]
       # Get the index of the item that matches the title
    idx = indices[id]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    item_indicies = [i[0] for i in sim_scores]

    # Return the top 10 most similar talks
    top_talks_idx = df['item'].iloc[item_indicies].index[:count]
    # get author, talk using top_talks_idx
    top_talks_author = df['author'].iloc[item_indicies].values[:count]
    top_talks_talk = df['talk'].iloc[item_indicies].values[:count]

    # create a result df 
    result_df = pd.DataFrame({'author': top_talks_author, 'talk': top_talks_talk})

    # rename columns
    result_df.columns = ['author', 'talk']

    return result_df



   # use indices to get 'author' and 'talk' columns from the dataframe

In [77]:
# get the first five recommendations using the funstion above
recomender('why good leaders make you feel safe', 5)

,author,talk
0,Rachel Botsman,weve stopped trusting institutions and started...
1,Simon Sinek,how great leaders inspire action
2,Marcos Aguiar,7 tools for building a business people trust
3,Jonathan Zittrain,the web as random acts of kindness
4,Frances Frei,how to build and rebuild trust
